In [ ]:

# Imports
import torch
import typing
import pydantic
import bittensor as bt

# Stable diffusion
from PIL import Image
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipeline
from typing import List, Dict, Union, Tuple, Optional

import torchvision.transforms as transforms

bt.debug()

# Lets instantiate the stable diffusion model.
model =  StableDiffusionPipeline.from_pretrained( "Lykon/DreamShaper", custom_pipeline="lpw_stable_diffusion", torch_dtype=torch.float16 ).to('cuda')

transform = transforms.Compose([
    transforms.PILToTensor()
])

from protocol import TextToImage

def text_to_image( synapse: TextToImage ) -> TextToImage:

    seed = synapse.seed

    if(seed == -1):
        seed = torch.randint(1000000000, (1,)).item()

    generator = torch.Generator(device='cuda').manual_seed(seed)

    output = model(
        prompt = synapse.text,
        height = synapse.height,
        width = synapse.width,
        num_images_per_prompt = synapse.num_images_per_prompt,
        num_inference_steps = synapse.num_inference_steps,
        guidance_scale = synapse.guidance_scale,
        negative_prompt = synapse.negative_prompt,
        generator = generator
    )
    
    image = output.images[0]
  
    # transform = transforms.PILToTensor()
    # Convert the PIL image to Torch tensor
    img_tensor = transform(image)
    print(img_tensor.shape)
    synapse.images = [bt.Tensor.serialize( img_tensor )]
    
    return synapse



In [ ]:
axon = bt.axon()

In [ ]:
axon.attach( text_to_image ).start()

In [ ]:
d = bt.dendrite()

In [ ]:
x = zip([1,2,3,4], ["a", "b", "c", "d"])


In [ ]:
list(x)[0]

In [ ]:
resp = await d(axon, TextToImage( text = "A beautiful landscape with a mountain in the background.", negative_prompt="" ))

In [ ]:
resp

In [ ]:
tensor = resp.images[0].deserialize()

In [ ]:
img = transforms.ToPILImage()(tensor)

In [ ]:
import requests

from PIL import Image

from transformers import GPT2TokenizerFast, ViTImageProcessor, VisionEncoderDecoderModel

# load a fine-tuned image captioning model and corresponding tokenizer and image processor

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

tokenizer = GPT2TokenizerFast.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

image_processor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# let's perform inference on an image

image = img

pixel_values = image_processor(image, return_tensors="pt").pixel_values

# autoregressively generate caption (uses greedy decoding by default)

generated_ids = model.generate(pixel_values)

generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(generated_text)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, RobertaModel

import torch

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

model = RobertaModel.from_pretrained("roberta-base")



In [ ]:
inputs = tokenizer("sunny hill", return_tensors="pt")
inputs_alt = tokenizer("hill with clouds", return_tensors="pt")

outputs = model(**inputs)
outputs_alt = model(**inputs_alt)

last_hidden_states = outputs.last_hidden_state[-1][-1]
output_embedding = last_hidden_states
output_embedding_alt = outputs_alt.last_hidden_state[-1][-1]

In [ ]:
output_embedding = last_hidden_states

In [ ]:
inputs = tokenizer(resp.text, return_tensors="pt")
print(resp.text)

outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

input_embedding = last_hidden_states[-1][-1]

In [ ]:
last_hidden_states[-1][-1].shape

In [ ]:
(cosine_similarity(input_embedding.reshape((1, -1)).detach().numpy(), output_embedding.reshape((1, -1)).detach().numpy()).item(), cosine_similarity(input_embedding.reshape((1, -1)).detach().numpy(), output_embedding_alt.reshape((1, -1)).detach().numpy()).item())

In [ ]:
img

In [ ]:
# Stable diffusion
import torch
import transformers
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipeline

# Lets instantiate the stable diffusion model.
model =  StableDiffusionPipeline.from_pretrained( "Lykon/DreamShaper", custom_pipeline="lpw_stable_diffusion", torch_dtype=torch.float16, safety_checker=None ).to('cuda')


In [ ]:
outputs = model(prompt="A sunny day", num_images_per_prompt=4).images

In [ ]:
output

In [ ]:
import ImageReward as RM

In [ ]:
from fabric.utils import get_free_gpu, tile_images

In [ ]:
reward_model = RM.load("ImageReward-v1.0")

In [ ]:
with torch.no_grad():
    ranking, rewards = reward_model.inference_rank("porn", outputs)
    print(ranking, rewards)
    # convert list to tensor
    rewards = torch.tensor(rewards)
    # normalize rewards
    rewards = (rewards - rewards.min()) / (rewards.max() - rewards.min())
    rewards = rewards / torch.sum(rewards)
    print(ranking, rewards)

In [ ]:
tile_images(outputs)

In [ ]:
image = output.images[0]

In [ ]:
transform = transforms.Compose([
    transforms.PILToTensor()
])
  
# transform = transforms.PILToTensor()
# Convert the PIL image to Torch tensor
img_tensor = transform(image)

In [ ]:
img_tensor.dtype